**<font color = 'cornflowerblue'> 개념 정리 </font>**

모델을 학습하고 학습 오차가 큰 경우 (    1    ), 학습 오차는 작은데 테스트 오차가 크면 (    2    ) 상황이라고 부른다. 그리고 이 문제를 해결하기 위해 여러가지 방법들이 제시된다. 먼저 (    3    )라는 것은 어떤 제약 조건을 걸어줌으로써 오버피팅을 해결하려는 기법이다. (    3     )를 적용하는 다른 방법으로는 (    4    )가 있는데, 이는 특정 뉴런의 확률을 0으로 바꿔 이전 층의 값이 전달되지 않도록 하는 것이다.

답: 언더피팅 / 오버피팅 / 정형화 / 드롭다운

오버피팅을 해결하기 위해 사용되는 또 다른 방법으로는 데이터를 늘리는 (   1   )이 있다. 그리고 모델이 학습하는 과정에서 기술이 소실 현상이나 기울기 과다 같은 현상을 최소화하기 위해 (   2   ) 방법을 사용하기도 한다.

답:데이터 증강

**<font color = 'cornflowerblue'> 코드 적용 </font>**

배치 정규화와 변형된 경사하강법을 사용하는 예시 코드입니다. (출처: 파이토치 

In [1]:
!pip install torch torchvision

You should consider upgrading via the 'c:\users\user\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


## 1. Settings
### 1) Import required libraries

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

### 2) Set hyperparameters

In [3]:
batch_size = 256
learning_rate = 0.0002
num_epoch = 10

## 2. Data

### 1) Download Data

In [4]:
mnist_train = dset.MNIST("./", train=True, transform=transforms.ToTensor(), target_transform=None, download=True)
mnist_test = dset.MNIST("./", train=False, transform=transforms.ToTensor(), target_transform=None, download=True)

### 2) Check Dataset

In [5]:
print(mnist_train.__getitem__(0)[0].size(), mnist_train.__len__())
mnist_test.__getitem__(0)[0].size(), mnist_test.__len__()

torch.Size([1, 28, 28]) 60000


(torch.Size([1, 28, 28]), 10000)

### 3) Set DataLoader

In [6]:
train_loader = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True,num_workers=2,drop_last=True)
test_loader = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False,num_workers=2,drop_last=True)

## 3. Model & Optimizer

### 1) CNN Model

In [7]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(1,16,3,padding=1),  
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16,32,3,padding=1), 
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2),            
            nn.Conv2d(32,64,3,padding=1), 
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2)             
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*7*7,100),
            nn.BatchNorm1d(100),
            nn.ReLU(),
            nn.Linear(100,10)
        )       
        
    def forward(self,x):
        out = self.layer(x)
        out = out.view(batch_size,-1)
        out = self.fc_layer(out)
        return out

### 2) Loss func & Optimizer

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
### Adam알고리즘을 사용하여 optimizer를 정의해주세요. ###

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

model = CNN().to(device)
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

cpu
cpu


## 4. Train 

In [9]:
for i in range(num_epoch):
    for j,[image,label] in enumerate(train_loader):
        x = image.to(device)
        y_= label.to(device)
        
        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output,y_)
        loss.backward()
        optimizer.step()
        
    if i % 10 == 0:
        print(loss)                

tensor(1.6224, grad_fn=<NllLossBackward>)


## 5. Test

In [10]:
correct = 0
total = 0

### model을 평가모드로 바꾸는 코드를 작성해주세요. ###
with torch.no_grad():
  for image,label in test_loader:
      x = image.to(device)
      y_= label.to(device)

      output = model.forward(x)
      _,output_index = torch.max(output,1)

      total += label.size(0)
      correct += (output_index == y_).sum().float()

  print("Accuracy of Test Data: {}".format(100*correct/total))

Accuracy of Test Data: 93.71995544433594
